In [1]:
import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

In [2]:
random.seed(30)

#### Load test dataset

In [3]:
# Root
from MyDataset import MyDataset

root = '../../train_val_test_dataset/IEEE_Case118/PG'

# Load all data with empty input
dataset = MyDataset(root=root, data_list=[])

In [4]:
# Get train dataset
train_size = 7000
test_size = 2000

test_dataset = dataset[-test_size:]

# Create train loader
############ Shuffle cannot be true for testing data ##############
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

#### Load trained model

In [5]:
from GNNPGRegression import GNNPGRegression
model = torch.load('./trained_model/PG_model.pt')
model.eval()

GNNPGRegression(
  (dropout): Dropout(p=0.3, inplace=False)
  (encoder): Sequential(
    (0): Linear(in_features=44, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=128, bias=True)
    (3): ReLU()
  )
  (gnn): ModuleList(
    (0): GCNConv(128, 128)
    (1): GCNConv(128, 128)
    (2): GCNConv(128, 128)
  )
  (decoder): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=24, bias=True)
    (3): ReLU()
  )
)

#### Model testing

In [6]:
with torch.no_grad():
    for i, loader in enumerate(test_loader):
        # Get ground true
        ground_true = loader.y.detach().numpy()
        ground_true = np.round(ground_true, 2)
        # Save ground true
        file_path = f'./model_evaluation/PG_true/true_{i+1}.csv'
        pd.DataFrame(ground_true).to_csv(file_path, columns=None, index=None, header=None)

        # Get prediction
        pred = model(loader.x, loader.edge_index, loader.edge_attr)
        pred = pred.detach().numpy()
        pred = np.round(pred, 2)
        # Save prediction
        file_path = f'./model_evaluation/PG_pred/pred_{i+1}.csv'
        pd.DataFrame(pred).to_csv(file_path, columns=None, index=None, header=None)